# How many ZEV are purchased through LCTOP & TIRCP? 
* An agency here is a recipient of LCTOP/TIRCP grants.

In [44]:
import A1_data_prep
import A2_tableau
import A6_zev
import A8_strings
import numpy as np
import pandas as pd
from babel.numbers import format_currency
from calitp_data_analysis.sql import to_snakecase


In [45]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

## LCTOP Clean Up 
* Subset for only zero emission adjacent work.

In [46]:
# Read in sheet
df_lctop = pd.read_excel(
    "gs://calitp-analytics-data/data-analyses/lctop/LCTOP_cleaned.xlsx",
    sheet_name="cleaned",
)

In [47]:
# Only keep certain cols
lctop_cols_to_keep = [
    "funding_year",
    "project_sub_type_ii",
    "project_description__short_",
    "puc_99313_funds",
    "puc_99314_funds",
    "total_project_request_99314_+_99313",
    "total_lctop_funds",
    "total_cci_funds",
    "total_project_cost",
    "status",
    "lead_agency",
]

In [48]:
# Subset df
df_lctop2 = df_lctop[lctop_cols_to_keep]

### Double Check Funding Column
* LCTOP funds column is not completely populated - what's the right column to use? 
* Double Checking that Total_project_request_99314_+_99313 is the "total awarded" column used in the LCTOP dashboards Courtney Williams emailed me

In [49]:
summary_lctop = df_lctop2.groupby(["funding_year"]).agg(
    {
        "total_project_request_99314_+_99313": "sum",
        "total_lctop_funds": "sum",
        "total_cci_funds": "sum",
    }
)

In [50]:
# Add grand total column
summary_lctop = summary_lctop.append(
    summary_lctop.sum(numeric_only=True), ignore_index=True
)

/tmp/ipykernel_1846/2835444261.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_lctop = summary_lctop.append(


In [51]:
# Format currency
for i in [
    "total_project_request_99314_+_99313",
    "total_lctop_funds",
    "total_cci_funds",
]:
    summary_lctop[i] = summary_lctop[i].apply(
        lambda x: format_currency(x, currency="USD", locale="en_US")
    )

In [52]:
# New column to differentiate award year
fy = ["14-15", "15-16", "16-17", "17-18", "18-19", "20-21", "Grand Total"]

In [53]:
# Add new col
summary_lctop["fiscal_year"] = fy

In [54]:
# Rearrange df
summary_lctop = summary_lctop[
    [
        "fiscal_year",
        "total_project_request_99314_+_99313",
        "total_lctop_funds",
        "total_cci_funds",
    ]
]

In [55]:
summary_lctop

,fiscal_year,total_project_request_99314_+_99313,total_lctop_funds,total_cci_funds
0,14-15,"$24,165,593.00",$0.00,$0.00
1,15-16,"$74,700,760.00",$0.00,$0.00
2,16-17,"$34,539,105.00","$87,455,156.00","$138,818,320.00"
3,17-18,"$96,864,564.50","$151,777,544.74","$266,660,391.74"
4,18-19,"$146,949,406.00","$213,867,091.00","$303,783,831.00"
5,20-21,"$146,054,354.00","$264,374,748.00","$313,451,971.00"
6,Grand Total,"$523,273,782.50","$717,474,539.74","$1,022,714,513.74"


### Filter ZEV adjacent projects.

In [56]:
df_lctop2["project_sub_type_ii"].value_counts()

New expanded/enhanced transit service                                 283
Free or reduced fares                                                 186
New transit related amenities                                         141
New zero-emission vehicles                                            123
Infrastructure to support zero-emission vehicle(s)                     34
New vehicles for new expanded/enhanced transit service                 29
Network/fare integration                                               24
Infrastructure to support new expanded/enhanced transit service        16
Renewable energy/fuel for transit service                               9
New Transit related amenities                                           3
Alternative transportation services                                     2
Vehicles upgrades to support new expanded/enhanced transit service      1
Name: project_sub_type_ii, dtype: int64

In [57]:
# df_lctop2.loc[df_lctop2['project_sub_type_ii'] == 'New expanded/enhanced transit service']

In [58]:
# Keep only zero emission adjacent project sub type ii
list_lctop_zev = [
    "New vehicles for new expanded/enhanced transit service",
    "New zero-emission vehicles",
]

In [59]:
# filter them all out at once.
df_lctop3 = df_lctop2[df_lctop2.project_sub_type_ii.isin(list_lctop_zev)]

In [60]:
# Make sure I filtered it correctly
df_lctop3["project_sub_type_ii"].value_counts()

New zero-emission vehicles                                123
New vehicles for new expanded/enhanced transit service     29
Name: project_sub_type_ii, dtype: int64

In [61]:
df_lctop3 = df_lctop3.reset_index()

In [62]:
# Extract an estimate of ZEV purchased per project and what type.
df_lctop3 = A6_zev.grab_zev_count(df_lctop3, "project_description__short_")

### Testing

In [63]:
zev_list= ['electric','CNG','zero emission','zero-emission','battery', 'hydrogen', 'hydrogen fuel cell', 'CELL']

In [64]:
# Filter out for zev projects
df_lctop_test= A8_strings.project_description_search(df_lctop2, "project_description__short_", zev_list)

/home/jovyan/data-analyses/tircp/A8_strings.py:289: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[description_column] = df[description_column].str.lower()
/home/jovyan/data-analyses/tircp/A8_strings.py:295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["keywords_flag_column"] = (


In [69]:
# Simplify project description to make it easier to grab numbers.
df_lctop_test = A8_strings.simplify_descriptions(
    df_lctop_test,
    "project_description__short_",
    "simple_description",
    A8_strings.description_words_to_delete,
)

In [70]:
keywords_list = [A8_strings.bus_list, A8_strings.lrv_list,
                 A8_strings.other_vehicles_list]

In [71]:
my_new_cols = ['buses','lrvs','other_vehicles']

In [72]:
df_lctop_test = A8_strings.total_procurement_estimates(df_lctop_test,
                        "simple_description", keywords_list, my_new_cols)
                                                     
    

In [73]:
for i in ['total_buses','total_lrvs','total_other_vehicles']:
    print(df_lctop_test[i].sum())

406
0
30


In [ ]:
# df_lctop_test[[  "project_description__short_",'simple_description','total_buses','total_lrvs','total_other_vehicles']]

### Answer why: <i>Didn’t seem like there was enough funding in LCTOP to support the procurement numbers that we’re showing</i>

In [75]:
def funding_vs_expenses(row):
    if row["total_project_request_99314_+_99313"] == row["total_project_cost"]:
        return "Fully funded"
    if row["total_project_cost"] == 0:
        return "No project cost info, just 0"
    elif row["total_project_request_99314_+_99313"] > row["total_project_cost"]:
        return "Funding exceeds total expenses"
    elif row["total_project_request_99314_+_99313"] < row["total_project_cost"]:
        return "Not fully funded"
    else:
        return "Not fully funded"

In [76]:
df_lctop3["fully_funded"] = df_lctop3.apply(lambda x: funding_vs_expenses(x), axis=1)

In [77]:
# Apply function to determine if a project is fully funded or not
df_lctop3["fully_funded"] = df_lctop3.apply(funding_vs_expenses, axis=1)

In [78]:
# Apply function to determine if a project is fully funded or not
df_lctop3["fully_funded"] = df_lctop3.apply(funding_vs_expenses, axis=1)

In [79]:
# Calculate percentage of funding
df_lctop3["percent_funded"] = (
    df_lctop3["total_project_request_99314_+_99313"] / df_lctop3["total_project_cost"]
).fillna(0)

In [80]:
df_lctop3 = df_lctop3.replace([np.inf, -np.inf], 0)

In [81]:
df_lctop3[
    [
        "funding_year",
        "total_project_request_99314_+_99313",
        "total_project_cost",
        "percent_funded",
        "fully_funded",
    ]
]

,funding_year,total_project_request_99314_+_99313,total_project_cost,percent_funded,fully_funded
0,14-15,34267.00,0.00,0.00,"No project cost info, just 0"
1,14-15,45703.00,0.00,0.00,"No project cost info, just 0"
2,14-15,107192.00,0.00,0.00,"No project cost info, just 0"
3,14-15,5784.00,0.00,0.00,"No project cost info, just 0"
4,14-15,69760.00,0.00,0.00,"No project cost info, just 0"
5,14-15,295041.00,0.00,0.00,"No project cost info, just 0"
6,15-16,112775.00,836000.00,0.13,Not fully funded
7,15-16,307569.00,307569.00,1.00,Fully funded
8,15-16,168281.00,5082000.00,0.03,Not fully funded
9,15-16,1127876.00,2466450.00,0.46,Not fully funded


In [82]:
# Raw numbers
df_lctop3["fully_funded"].value_counts()

Not fully funded                138
Fully funded                      8
No project cost info, just 0      6
Name: fully_funded, dtype: int64

In [83]:
# Percentage of projects by funding status
df_lctop3["fully_funded"].value_counts() * 100 / len(df_lctop3)

Not fully funded               90.79
Fully funded                    5.26
No project cost info, just 0    3.95
Name: fully_funded, dtype: float64

In [84]:
df_lctop3["percent_funded"].median(), df_lctop3["percent_funded"].mean()

(0.1511081301148903, 0.2579673906528877)

## TIRCP Clean Up

In [85]:
df_tircp = to_snakecase(A2_tableau.tableau_dashboard())

/opt/conda/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [86]:
# Subset for only cols of interest
tircp_columns = [
    "award_year",
    "grant_recipient",
    "title",
    "description",
    "tircp",
    "expended_amount",
    "expended_percent",
    "progress",
]

In [87]:
df_tircp2 = df_tircp[tircp_columns]

In [89]:
df_tircp2= A8_strings.project_description_search(df_tircp2, "description", zev_list)

/home/jovyan/data-analyses/tircp/A8_strings.py:289: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[description_column] = df[description_column].str.lower()
/home/jovyan/data-analyses/tircp/A8_strings.py:295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["keywords_flag_column"] = (


In [91]:
# Extract an estimate of ZEV purchased per project and what type.
df_tircp3 = A6_zev.grab_zev_count(df_tircp2, "description")

In [92]:
# Simplify project description to make it easier to grab numbers.
df_tircp_test = A8_strings.simplify_descriptions(
    df_tircp2,
    "description",
    "simple_description",
    A8_strings.description_words_to_delete,
)

In [95]:
df_tircp_test.columns

Index(['index', 'award_year', 'grant_recipient', 'title', 'description',
       'tircp', 'expended_amount', 'expended_percent', 'progress',
       'keywords_flag_column', 'number_of_zev', 'lrv_or_bus',
       'simple_description', 'total_buses', 'total_lrvs',
       'total_other_vehicles', 'buses', 'lrvs', 'other_vehicles'],
      dtype='object')

In [94]:
df_tircp_test = A8_strings.total_procurement_estimates(df_tircp_test,
                        "simple_description", keywords_list, my_new_cols)
                                                     
    

ValueError: columns overlap but no suffix specified: Index(['total_buses'], dtype='object')

In [ ]:
# df_tircp_test[['description','simple_description','total_buses','total_lrvs','total_other_vehicles']]

In [ ]:
for i in ['total_buses','total_lrvs','total_other_vehicles']:
    print(df_tircp_test[i].sum())

In [ ]:
# df_tircp_zev.head(5)

## Manual Clean Up
* Although I extracted integers from the description columns to find the number of ZEV purchased, some of the values were not populated/correct.

In [ ]:
# Export LCTOP and TIRCP to fill in the # of zev buses manually for any rows that didn't pick up the number
"""
with pd.ExcelWriter(
    "gs://calitp-analytics-data/data-analyses/tircp/LCTOP_ZEV.xlsx"
) as writer:
    df_lctop3.to_excel(writer, sheet_name="lctop", index=True)
    df_tircp_zev.to_excel(writer, sheet_name="tircp", index=True)
    """

## Analysis

In [96]:
# Open up workbook that I manually filled in
# Sheets I want
sheets_list = ["lctop", "tircp"]

# Open the workbook in a dictionary
dict_df1 = pd.read_excel(
    "gs://calitp-analytics-data/data-analyses/tircp/LCTOP_TIRCP_ZEV_manual.xlsx",
    sheet_name=sheets_list,
)

In [97]:
# Grab each sheet
lctop_clean = to_snakecase(dict_df1.get("lctop"))
tircp_clean = to_snakecase(dict_df1.get("tircp"))

In [98]:
tircp_clean.head(2)

,award_year,grant_recipient,title,description,tircp,expended_amount,expended_percent,progress,number_of_zev,lrv_or_bus
0,2022,Anaheim Transportation Network,ATN FAST (Family of Advanced Solutions for Transit): Revolutionizing Transit for a Global Audience,"creates a zero-emission transit ecosystem that offers end-to-end solutions for residents, employees and the global audience drawn by tourism/convention centers and the la 2028 summer olympics events. project components include (1) purchase of 7 zero-emission battery electric vans to implement a new service connecting john wayne airport to anaheim, (2) purchase of 10 electric vehicles and associated infrastructure to expand on-demand micro transit services into new neighborhoods and service areas, (3) purchase of 15 zeroemission buses to replace existing buses and augment existing routes, including installation of photovoltaic electricity generation at 2 facilities, and (4) purchase of 10 additional zero-emission buses for a new east/west connector service.",0,0.00,0.00,No expenditures recorded,42,bus
1,2018,Santa Barbara County Association Of Governments,Goleta Train Depot,"improves the transit facility for bus, train, bicycle and pedestrians by constructing a modern, multi-modal train station and provides a safe, functional and inviting facility that accommodates improved bus transit service, shuttles from santa barbara airport and the university of california santa barbara. includes a zero emission shuttle bus to serve the santa barbara airport and improvement in local bus service providing station access to ab 1550 communities. project award includes $250,000 of funding to address ne2rk integration opportunities, including development of improved connections to other rail and transit services, and to identify opportunities for ab 1550 benefits. these efforts will be coordina facchinited with tircp 28 projects awarded to sbcag for the clean air express and lossan for the improvement and expansion of service between los angeles and san luis obispo.",13009000,1552456.00,0.45,On Track,1,bus


### TIRCP

In [99]:
tircp_sum = ["tircp", "number_of_zev"]
tircp_count = ["title"]
tircp_group = ["award_year"]
tircp_mon = ["tircp"]

In [102]:
# Create summary table for all ZEV projects
tircp_summary = A6_zev.zev_summary(
    tircp_clean, df_tircp, tircp_group, tircp_sum, tircp_count, tircp_mon
)

/home/jovyan/data-analyses/tircp/A6_zev.py:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  m1 = m1.append(m1.sum(numeric_only=True), ignore_index=True)


In [103]:
# Calculate out projects that are ZEV vs total projects
tircp_summary["Percent of Projects that are ZEV Adjacent"] = (
    tircp_summary["Title X"] / tircp_summary["Title Y"]
)

In [104]:
# Clean columns
tircp_summary = tircp_summary.rename(
    columns={
        "Title X": "Total ZEV Projects",
        "Number Of Zev": "Total ZEV",
        "Title Y": "Total Projects in Cycle",
        "Tircp": "Total Amount Awarded",
    }
)

In [105]:
# Rename Award Year to Cycles
tircp_summary["Award Year"] = tircp_summary["Award Year"].replace(
    {
        2015: "Cycle 1",
        2016: "Cycle 2",
        2018: "Cycle 3",
        2020: "Cycle 4",
        2022: "Cycle 5",
    }
)

In [106]:
# Edit so Grand total row is obvious
tircp_summary["Award Year"].loc[5] = "Grand Total"

/tmp/ipykernel_1846/3311859722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tircp_summary["Award Year"].loc[5] = "Grand Total"


In [107]:
# Manually calclate out total percent of ZEV projects
tircp_summary["Percent of Projects that are ZEV Adjacent"].loc[5] = (
    tircp_summary["Total ZEV Projects"].sum()
    / tircp_summary["Total Projects in Cycle"].sum()
)

/tmp/ipykernel_1846/2407361157.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tircp_summary["Percent of Projects that are ZEV Adjacent"].loc[5] = (


In [108]:
tircp_summary

,Award Year,Total Amount Awarded,Total ZEV,Total ZEV Projects,Total Projects in Cycle,Percent of Projects that are ZEV Adjacent
0,Cycle 1,$74745000.0,38,4,14,0.29
1,Cycle 2,$41930000.0,2,3,14,0.21
2,Cycle 3,$385742000.0,205,13,28,0.46
3,Cycle 4,$247475000.0,100,9,17,0.53
4,Cycle 5,$0.0,434,15,23,0.65
5,Grand Total,$749892000.0,779,44,96,0.46


In [111]:
A6_zev.basic_bar_chart(
    tircp_summary.iloc[:-1],
    "Award Year",
    "Total ZEV Projects",
    "Award Year",
    "Total ZEV Projects by FY (TIRCP)",
)

alt.Chart(...)

In [112]:
A6_zev.basic_bar_chart(
    tircp_summary.iloc[:-1],
    "Award Year",
    "Total ZEV",
    "Award Year",
    "Total ZEV to be or already Purchased by FY (TIRCP)",
)

alt.Chart(...)

#### How many  ZEV already purchased/in the pipeline?

In [113]:
# Find out how many buses already purchased by looking at only projects with 100% of allocated amount spent.
tircp_zev_done = tircp_clean.loc[
    tircp_clean["progress"] == "100% of allocated funds spent"
]
f"Approximately {tircp_zev_done.number_of_zev.sum()} ZEV already purchased and ${tircp_zev_done.tircp.sum()} spent"

'Approximately 75 ZEV already purchased and $108711000 spent'

In [114]:
# Look at projects that are done - says 0 when number of buses is unknown.
tircp_zev_done = tircp_zev_done[
    ["award_year", "grant_recipient", "number_of_zev", "tircp"]
].sort_values("number_of_zev", ascending=False)

tircp_zev_done["tircp"] = tircp_zev_done["tircp"].apply(
    lambda x: format_currency(x, currency="USD", locale="en_US")
)

In [115]:
tircp_zev_done

,award_year,grant_recipient,number_of_zev,tircp
6,2018,Anaheim Transportation Network,40,"$28,617,000.00"
11,2015,Antelope Valley Transit Authority,13,"$24,403,000.00"
13,2015,San Joaquin Regional Transit District,12,"$6,841,000.00"
26,2015,Orange County Transportation Authority,5,"$2,320,000.00"
27,2018,Santa Barbara County Association Of Governments,5,"$9,600,000.00"
38,2016,Antelope Valley Transit Authority,0,"$8,930,000.00"
39,2016,Orange County Transportation Authority,0,"$28,000,000.00"


#### How many  ZEV in the pipeline?

In [116]:
tircp_zev_pipeline = tircp_clean.loc[
    tircp_clean["progress"] != "100% of allocated funds spent"
]
f"Approximately {tircp_zev_pipeline.number_of_zev.sum()} ZEV in the pipeline to be purchased and ${tircp_zev_pipeline.tircp.sum()} in projected funds."

'Approximately 704 ZEV in the pipeline to be purchased and $641181000 in projected funds.'

In [117]:
tircp_zev_pipeline = (
    tircp_zev_pipeline.groupby(["grant_recipient", "award_year"])
    .agg({"number_of_zev": "sum", "tircp": "sum"})
    .reset_index()
    .sort_values("number_of_zev", ascending=False)
)

In [118]:
tircp_zev_pipeline["tircp"] = tircp_zev_pipeline["tircp"].apply(
    lambda x: format_currency(x, currency="USD", locale="en_US")
)

In [119]:
tircp_zev_pipeline

,grant_recipient,award_year,number_of_zev,tircp
20,Los Angeles County Metropolitan Transportation,2022,261,$0.00
9,City Of Los Angeles,2018,112,"$36,104,000.00"
17,La County Metropolitan Transportation Authority,2020,60,"$107,050,000.00"
1,Anaheim Transportation Network,2022,42,$0.00
33,Sonoma County Transportation Authority,2022,30,$0.00
7,City Of Glendale And Arroyo Verdugo Communities,2022,27,$0.00
36,Tulare County Regional Transit Agency,2022,14,$0.00
32,Solano Transportation Authority,2018,13,"$10,788,000.00"
5,City Of Cupertino,2022,12,$0.00
3,Antelope Valley Transit Authority,2020,11,"$6,503,000.00"


#### ZEV Breakdown

In [120]:
tircp_zev_breakout = tircp_clean.groupby(["lrv_or_bus"]).agg(
    {"tircp": "sum", "number_of_zev": "sum"}
)

In [121]:
tircp_zev_breakout["tircp"] = tircp_zev_breakout["tircp"].apply(
    lambda x: format_currency(x, currency="USD", locale="en_US")
)

In [122]:
tircp_zev_breakout

,tircp,number_of_zev
lrv_or_bus,,
bus,"$274,879,000.00",694
light rail vehicle,"$41,181,000.00",8
other,"$269,310,000.00",77
rail,"$164,522,000.00",0


### LCTOP

In [123]:
lctop_count = ["project_sub_type_ii"]
lctop_group = ["funding_year"]
lctop_sum = [
    "total_project_request_99314_+_99313",
    "total_lctop_funds",
    "number_of_zev",
]
lctop_monetary = [
    "total_project_request_99314_+_99313",
    "total_lctop_funds",
]

In [125]:
# Create summary table for all ZEV projects
lctop_summary = A6_zev.zev_summary(
    lctop_clean, df_lctop2, lctop_group, lctop_sum, lctop_count, lctop_monetary
)

/home/jovyan/data-analyses/tircp/A6_zev.py:118: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  m1 = m1.append(m1.sum(numeric_only=True), ignore_index=True)


In [126]:
# Calculate out projects that are ZEV vs total projects
lctop_summary["Percent of Projects that are ZEV Adjacent"] = (
    lctop_summary["Sub Type Ii X"] / lctop_summary["Sub Type Ii Y"]
)

In [127]:
# Clean columns
lctop_summary = lctop_summary.rename(
    columns={
        "Number Of Zev": "Total ZEV",
        "Sub Type Ii X": "Total ZEV Projects",
        "Sub Type Ii Y": "Total Projects in Cycle",
        "Total  Request 99314 + 99313": "Total Amount Awarded",
    }
)

In [128]:
# Change certain values from float to int
lctop_summary[
    ["Total ZEV", "Total ZEV Projects", "Total Projects in Cycle"]
] = lctop_summary[
    ["Total ZEV", "Total ZEV Projects", "Total Projects in Cycle"]
].astype(
    "int64"
)

In [129]:
# Make the grand total row more obvious
lctop_summary["Funding Year"].loc[6] = "Grand Total"

In [130]:
# Manually calc out total ZEV projects
lctop_summary["Percent of Projects that are ZEV Adjacent"].loc[6] = (
    lctop_summary["Total ZEV Projects"].sum()
    / lctop_summary["Total Projects in Cycle"].sum()
)

In [131]:
# lctop_summary

In [133]:
A6_zev.basic_bar_chart(
    lctop_summary.iloc[:-1],
    "Funding Year",
    "Total ZEV",
    "Funding Year",
    "Total ZEV to be or already Purchased by FY (LCTOP)",
)

alt.Chart(...)

In [134]:
A6_zev.basic_bar_chart(
    lctop_summary.iloc[:-1],
    "Funding Year",
    "Total ZEV Projects",
    "Funding Year",
    "Total ZEV Projects by FY (LCTOP)",
)

alt.Chart(...)

#### ZEV in the pipeline
* Status column as shown below has only one "closed", 50 null values, and 101 open.

In [135]:
lctop_clean["status"].value_counts()

open    93
None    32
Name: status, dtype: int64

In [136]:
lctop_done = lctop_clean.loc[lctop_clean["status"] == "closed"]
f"Approximately {lctop_done.number_of_zev.sum()} ZEV already purchased and ${lctop_done['total_project_request_99314_+_99313'].sum()} spent"

'Approximately 0 ZEV already purchased and $0.0 spent'

In [137]:
lctop_pipeline = lctop_clean.loc[lctop_clean["status"] != "closed"]
f"Approximately {lctop_clean.number_of_zev.sum()} ZEV to be purchased and ${lctop_clean['total_project_request_99314_+_99313'].sum()} in projected funds"

'Approximately 668 ZEV to be purchased and $92513588.5 in projected funds'

#### All Applicants

In [138]:
lctop_clean["total_project_request_99314_+_99313"] = lctop_clean[
    "total_project_request_99314_+_99313"
].astype("int64")

In [139]:
lctop_applicants = (
    lctop_clean.groupby(["funding_year", "lead_agency"])
    .agg({"number_of_zev": "sum", "total_project_request_99314_+_99313": "sum"})
    .reset_index()
    .sort_values("number_of_zev", ascending=False)
)

In [140]:
lctop_applicants["total_project_request_99314_+_99313"] = lctop_applicants[
    "total_project_request_99314_+_99313"
].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))

In [141]:
# lctop_applicants

#### ZEV Breakdown

In [142]:
# lctop_clean.loc[lctop_clean["lrv_or_bus"] == 'other']

In [143]:
lctop_zev_breakout = lctop_clean.groupby(["lrv_or_bus"]).agg(
    {"total_project_request_99314_+_99313": "sum", "number_of_zev": "sum"}
)

In [144]:
lctop_zev_breakout = lctop_zev_breakout.rename(
    columns={"total_project_request_99314_+_99313": "awarded"}
)

In [145]:
lctop_zev_breakout["awarded"] = lctop_zev_breakout["awarded"].apply(
    lambda x: format_currency(x, currency="USD", locale="en_US")
)

In [146]:
lctop_zev_breakout

,awarded,number_of_zev
lrv_or_bus,,
bus,"$74,531,027.00",597
other,"$10,102,813.00",67
rail,"$7,879,748.00",4


## Export

In [147]:
"""
with pd.ExcelWriter(
    "gs://calitp-analytics-data/data-analyses/tircp/LCTOP_ZEV_final.xlsx"
) as writer:
    tircp_summary.to_excel(writer, sheet_name="tircp_summary", index=False)
    tircp_zev_done.to_excel(writer, sheet_name="tircp_finished_projects", index=False)
    tircp_zev_pipeline.to_excel(writer, sheet_name="tircp_inprogress", index=False)
    lctop_summary.to_excel(writer, sheet_name="lctop_summary", index=False)
    lctop_applicants.to_excel(writer, sheet_name="lctop_applicants", index=False)
    """

'\nwith pd.ExcelWriter(\n    "gs://calitp-analytics-data/data-analyses/tircp/LCTOP_ZEV_final.xlsx"\n) as writer:\n    tircp_summary.to_excel(writer, sheet_name="tircp_summary", index=False)\n    tircp_zev_done.to_excel(writer, sheet_name="tircp_finished_projects", index=False)\n    tircp_zev_pipeline.to_excel(writer, sheet_name="tircp_inprogress", index=False)\n    lctop_summary.to_excel(writer, sheet_name="lctop_summary", index=False)\n    lctop_applicants.to_excel(writer, sheet_name="lctop_applicants", index=False)\n    '